In [6]:
from ema_workbench import MultiprocessingEvaluator
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress, ArchiveLogger)
from ema_workbench.em_framework.optimization import (EpsNSGAII)
import numpy as np

if __name__ == "__main__":

    # --- Load your model ---
    model, planning_steps = get_model_for_problem_formulation(2)

    # --- Set reference scenario (e.g., median or default values for all uncertainties) ---
    reference_scenario = {unc.name: (unc.dist.mean() if hasattr(unc.dist, 'mean') else 0.5)
                          for unc in model.uncertainties}

    # --- Optimization setup ---
    convergence_metrics = [HyperVolume(minimum=[0, 0], maximum=[1, 1]),
                           EpsilonProgress()]

    nfe = 1000  # Number of function evaluations (you can increase this)

    with MultiprocessingEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(
            nfe=nfe,
            searchover='levers',
            epsilons=[0.05, 0.05],  # One epsilon per objective
            convergence=convergence_metrics,
            reference=reference_scenario,
            algorithm=EpsNSGAII
        )

    # --- Save and inspect results ---
    import pandas as pd
    pd.DataFrame(results).to_csv("step2_mordm_results.csv", index=False)

    # Optional: visualize the trade-offs
    from ema_workbench.analysis import parcoords
    parcoords.ParallelCoordinatesFigure(results).show()

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] terminating pool


AssertionError: 

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from ema_workbench import (Model, Policy, MultiprocessingEvaluator, Samplers)
from ema_workbench.analysis import feature_scoring
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.analysis import feature_scoring